In [67]:
import pandas as pd
import matplotlib.pyplot as plt
data = pd.read_csv("zomato_data.csv")

In [63]:
data.head()

,Unnamed: 0,NAME,PRICE,CUSINE_CATEGORY,CITY,REGION,URL,PAGE NO,CUSINE TYPE,TIMING,RATING_TYPE,RATING,VOTES
0,0,The Salt Cafe Kitchen & Bar,1000,"North Indian,Continental,Italian",Agra,Tajganj,https://www.zomato.com/agra/the-salt-cafe-kitc...,1,Casual Dining,1130am to 1130pm(Mon-Sun),Excellent,4.8,273
1,1,Bhagat Halwai,300,"Desserts,Bakery,Fast Food,South Indian",Agra,Civil Lines,https://www.zomato.com/agra/bhagat-halwai-civi...,1,Quick Bites,730am to 11pm(Mon-Sun),Very Good,4.3,554
2,2,Bikanervala,700,"North Indian,South Indian,Mithai,Street Food,D...",Agra,Khandari,https://www.zomato.com/agra/bikanervala-khanda...,1,Quick Bites,8am to 11pm(Mon-Sun),Very Good,4.4,645
3,3,Mama Chicken Mama Franky House,600,"North Indian,Mughlai,Rolls,Chinese,Fast Food,S...",Agra,Agra Cantt,https://www.zomato.com/agra/mama-chicken-mama-...,1,Quick Bites,"1230pm to 12midnight(Mon,Wed,Thu,Fri,Sat...",Very Good,4.3,894
4,4,Honeydew Restaurant,500,"North Indian,Fast Food,Chinese,South Indian",Agra,Dayal Bagh,https://www.zomato.com/agra/honeydew-restauran...,1,Quick Bites,"11am to 1035pm(Mon,Tue,Wed,Fri,Sat),11am to ...",Very Good,4.1,489


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105623 entries, 0 to 105622
Data columns (total 13 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   Unnamed: 0       105623 non-null  int64 
 1   NAME             105623 non-null  object
 2   PRICE            105623 non-null  int64 
 3   CUSINE_CATEGORY  105558 non-null  object
 4   CITY             105623 non-null  object
 5   REGION           105623 non-null  object
 6   URL              105623 non-null  object
 7   PAGE NO          105623 non-null  int64 
 8   CUSINE TYPE      105623 non-null  object
 9   TIMING           104982 non-null  object
 10  RATING_TYPE      105623 non-null  object
 11  RATING           105623 non-null  object
 12  VOTES            105623 non-null  object
dtypes: int64(3), object(10)
memory usage: 10.5+ MB


#### I am using price column of zomato dataset to apply data smoothing and data transformation techniques

we can see that price data is highly left skewed

In [19]:
import statistics as st 
def stats(price):
    minm = min(price)
    maxm = max(price)
    median = st.median(price)
    mean = st.mean(price)
    pstdev = st.pstdev(price)
    return minm,maxm,median,mean,pstdev
min_price,max_price,median_price,mean_price,pstdev_price = stats(data["PRICE"])

## Data smoothing

In [98]:
def bin_mean_smoothing(data,bin_size=5):
    bin_mean = []
    for iterator in range(0,len(data),bin_size):
        data_mean = st.mean(data[iterator:iterator+bin_size])
        for i in range(bin_size):
            bin_mean.append(data_mean)
    return bin_mean

In [94]:
def bin_median_smoothing(data,bin_size=5):
    bin_median = []
    for iterator in range(0,len(data),bin_size):
        data_median = st.median(data[iterator:iterator+bin_size])
        for i in range(bin_size):
            bin_median.append(data_median)
    return bin_median

In [53]:
def boundary_bin_smoothing(data,bin_size=5):
    bin_boundary = []
    for iterator in range(0,len(data),bin_size):
        half_bin_size = int(bin_size/2)
        data_min = min(data[iterator:iterator+bin_size])
        data_max = max(data[iterator:iterator+bin_size])
        for i in range(half_bin_size):
            bin_boundary.append(data_min)
        for i in range(half_bin_size+1):
            bin_boundary.append(data_max)
    return bin_boundary

In [99]:
data_price = data["PRICE"].sort_values()
bin_mean = bin_mean_smoothing(data_price)
bin_median = bin_median_smoothing(data_price)
bin_boundary = boundary_bin_smoothing(data_price)

## Data Transformation

In [112]:
def min_max_trans(data):
    min_max_tran = []
    minm = min(data)
    maxm = max(data)
    for iterator in range(0,len(data)):
        min_max_tran.append((data[iterator]-minm)/(maxm-minm))
    return min_max_tran

In [106]:
def z_score_trans(data):
    z_score = []
    mean = st.mean(data)
    pstdev = st.pstdev(data)
    for iterator in range(0,len(data)):
        z_score.append((data[iterator])-mean/pstdev)
    return z_score

In [113]:
data_price = data["PRICE"]
min_max_price = min_max_trans(data_price)
z_score_price= z_score_trans(data_price)